In [1]:
import numpy as np
import pandas as pd
import pandas_ta as ta

In [2]:
def simple_moving_average(df, window):
    df2 = df.copy()
    df2['MA'] = df2['Close'].rolling(window=window).mean()
    return df2
def z_score(df, window):
    df2 = df.copy()
    
    df2['Z Score'] = (df2['Close'] - df2['MA']) / df2['Close'].rolling(window=window).std()
    return df2
def rsi(df, column, window = 14):
    delta = df[column].diff()
    plus = delta.where(delta > 0, 0)
    loss = delta.where(delta < 0, 0)
    avg_p = plus.rolling(window=window, min_periods=1).mean()
    avg_l = loss.rolling(window=window, min_periods=1).mean()
    rs = avg_p/avg_l
    rsi = 100 - (100 / (1 + rs))
    return rsi


In [ ]:
def mean_reversion_strategy(df):
    df2 = df.copy()
    df2['RSI'] = rsi(df2, 'Close')
    df2['Z Score'] = z_score(df2, 'Close')
    df2['MA'] = simple_moving_average(df2, 'Close')
    df2['Signal'] = np.where(df2['Z Score'] > 1, 1, 0)
    df2['Position'] = df2['Signal'].diff()
    return df2

def backtest(df, strategy):
    df2 = df.copy()
    df2['Position'] = strategy(df2)
    df2['Return'] = df2['Close'].pct_change() * df2['Position'].shift(1)
    df2['Cumulative Return'] = (1 + df2['Return']).cumprod() - 1
    return df2